## **Fase 6 - Machine Learning: Construcción y Evaluación del Modelo Predictivo**
*En esta fase, nos centramos en construir un modelo predictivo que sea capaz de estimar con precisión el **precio por noche** de las propiedades. Este paso es crucial para convertir los datos procesados en un conocimiento valioso, ya que nos permitirá hacer predicciones sobre el precio de futuras propiedades a partir de sus características. A través de técnicas de machine learning avanzadas, buscamos encontrar el modelo más eficiente, que no solo sea preciso, sino también interpretativo y generalizable*

A lo largo de esta fase, aplicaremos una serie de algoritmos y evaluaremos su rendimiento, seleccionando aquel que mejor se ajuste a nuestras necesidades. Además, dedicaremos tiempo a ajustar los parámetros del modelo para maximizar su capacidad predictiva, asegurando así que sea robusto y confiable.
Con este enfoque, buscamos no solo predecir el precio de forma precisa, sino también proporcionar una comprensión profunda de los factores clave que afectan los precios de las propiedades, lo que puede ser útil en la toma de decisiones empresariales y en el desarrollo de futuras investigaciones.

In [2]:
# General
import pandas as pd
import numpy as np
import joblib

# Escaladores
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Train, Test
from sklearn.model_selection import train_test_split

# Modelos
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

# Neural Neutworks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Hiperparametrización
from sklearn.model_selection import GridSearchCV

# Métricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

In [2]:
df_encoded = pd.read_csv('data\df_processed_to_ML.csv')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\diogo\AppData\Local\Temp\ipykernel_5640\1166888792.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_encoded = pd.read_csv('data\df_processed_to_ML.csv')


In [3]:
df_encoded

,prices_per_night,ratings,cleaning_fee,dormitorios,camas,baños,maximum_guests,check_in_hour,check_out_hour,total_hours_checkin,...,dormitorio y lavandería,entretenimiento,exterior,internet y oficina,para familias,privacidad y seguridad,seguridad en el hogar,servicios,habitacion,alojamiento entero
0,115.0,0.00,0.0,1.0,1.0,1.0,2.0,900.0,720.0,9.0,...,7.0,1.0,2.0,1.0,0.0,0.0,1.0,4.0,0.0,0.0
1,46.0,0.00,15.0,1.0,1.0,0.5,1.0,1020.0,660.0,7.0,...,6.0,0.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,0.0
2,47.0,4.66,0.0,1.0,1.0,0.5,1.0,900.0,720.0,9.0,...,8.0,1.0,1.0,1.0,0.0,1.0,0.0,5.0,1.0,0.0
3,100.0,4.89,35.0,1.0,1.0,1.0,1.0,960.0,720.0,8.0,...,10.0,10.0,3.0,2.0,2.0,0.0,5.0,4.0,0.0,0.0
4,33.0,4.40,0.0,1.0,1.0,0.5,1.0,900.0,660.0,9.0,...,4.0,0.0,1.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,55.0,4.74,10.0,1.0,1.0,0.5,3.0,900.0,660.0,9.0,...,7.0,1.0,0.0,2.0,0.0,3.0,0.0,1.0,1.0,0.0
2493,60.0,4.78,0.0,1.0,1.0,0.5,2.0,900.0,600.0,9.0,...,4.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
2494,104.0,4.96,0.0,2.0,3.0,2.0,4.0,900.0,720.0,9.0,...,9.0,1.0,0.0,2.0,0.0,0.0,2.0,3.0,0.0,0.0
2495,120.0,4.83,50.0,1.0,1.0,1.0,2.0,900.0,660.0,9.0,...,8.0,3.0,3.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0


**Train Test Split**

In [4]:
X = df_encoded.drop("prices_per_night", axis = 1)
y = df_encoded["prices_per_night"]
print(X.shape, y.shape)

(2497, 25) (2497,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

X_train: (1997, 25), y_train: (1997,)
X_test: (500, 25), y_test: (500,)


In [ ]:
# Guardar los nombres originales de las columnas para usar en streamlit
#original_columns = X.columns.tolist()

['ratings',
 'cleaning_fee',
 'dormitorios',
 'camas',
 'baños',
 'maximum_guests',
 'check_in_hour',
 'check_out_hour',
 'total_hours_checkin',
 'log_num_reviews',
 'aparcamiento e instalaciones',
 'baño',
 'calefacción y refrigeración',
 'características de la ubicación',
 'cocina y comedor',
 'dormitorio y lavandería',
 'entretenimiento',
 'exterior',
 'internet y oficina',
 'para familias',
 'privacidad y seguridad',
 'seguridad en el hogar',
 'servicios',
 'habitacion',
 'alojamiento entero']

**Escaladores**

In [6]:
x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = y_scaler.transform(np.array(y_test).reshape(-1, 1))

In [7]:
# Salvar los escaladores para usar en streamlit
joblib.dump(x_scaler, 'data/pkl/x_scaler.pkl')
joblib.dump(y_scaler, 'data/pkl/y_scaler.pkl')

['data/pkl/y_scaler.pkl']

**Selección de los Modelos**
- Evaluaremos cada modelo mediante las **métricas de rendimiento**, tales como el **Error Cuadrático Medio (RMSE)** y el **R^2**, con el fin de seleccionar el que brinde el mejor rendimiento predictivo.

In [9]:
# Definimos los modelos
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Support Vector Regressor": SVR(),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "MLP Regressor": MLPRegressor(random_state=42)
}

In [10]:
# Lista para almacenar los resultados
resultados_lista = []

# Bucle para entrenar cada modelo y calcular métricas
for model_name, model in models.items():
    model.fit(X_train, y_train.ravel())
    
    y_hat = model.predict(X_test)
    
    # Desescalado de las predicciones
    y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()
    y_hat_inv = y_scaler.inverse_transform(y_hat.reshape(-1, 1)).ravel()
    
    # Cálculo de métricas
    mae = mean_absolute_error(y_test_inv, y_hat_inv)
    mse = mean_squared_error(y_test_inv, y_hat_inv)
    rmse = root_mean_squared_error(y_test_inv, y_hat_inv)
    r2 = r2_score(y_test_inv, y_hat_inv)
    
    # Almacenar resultados
    resultados_lista.append({
        "model_name": model_name,
        "mae": mae,
        "mse": mse,
        "rmse" : rmse,
        "r2_score": r2
    })



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 582
[LightGBM] [Info] Number of data points in the train set: 1997, number of used features: 24
[LightGBM] [Info] Start training from score 0.143845


In [11]:
# Crear DataFrame y ordenar por r2_score
resultados = pd.DataFrame(resultados_lista)
resultados = resultados.sort_values(by="r2_score", ascending=False)
#Creamos el pickle
resultados.to_pickle("data/pkl/resultados_modelos.pkl")

In [12]:
resultados

,model_name,mae,mse,rmse,r2_score
5,LightGBM,9.727440,244.834744,15.647196,0.860073
1,Random Forest,8.654486,265.400936,16.291131,0.848319
4,XGBoost,8.297636,292.500981,17.102660,0.832831
3,Gradient Boosting,13.716214,376.139506,19.394316,0.785031
0,Linear Regression,17.901984,615.842171,24.816168,0.648037
6,MLP Regressor,19.023449,636.524040,25.229428,0.636217
2,Support Vector Regressor,20.278318,652.793475,25.549823,0.626919


**Ajuste de Hiperparámetros**
   - Para maximizar el rendimiento del modelo, ajustaremos los hiperparámetros clave utilizando **Grid Search**. Este paso es crucial para obtener el mejor modelo posible para nuestros datos.

In [13]:
lgb = LGBMRegressor(random_state=42) # Quizas el objective se pone aquí
param_grid = {
    'n_estimators': [100, 500, 1000],
    'num_leaves': [31, 63, 127],
    'max_depth': [5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_samples': [20, 50],
    'subsample': [0.7, 0.8],
    'reg_lambda': [0, 1],
    'reg_alpha': [0, 1],
    'boosting_type': ['gbdt', 'dart'],
    'objective': ['regression']
}

In [14]:
grid_search = GridSearchCV(estimator=lgb, param_grid=param_grid, 
                           scoring='neg_mean_absolute_error', cv=3, 
                           verbose=1, n_jobs=-1)

In [15]:
%%time

grid_search.fit(X_train, y_train)
# Mostramos los mejores parámetros y resultados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# comprobamos los  mejores parámetros
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculamos y mostramos las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE", rmse)
print("R²:", r2)

Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


KeyboardInterrupt: 

Creación del pickle

In [ ]:
# Creamos un diccionario con los resultados
grid_search = {
    "best_params": best_params,
    "mae": mae,
    "mse": mse,
    "rmse": rmse,
    "r2_score": r2
}

grid_search = pd.DataFrame(grid_search)
grid_search = grid_search.sort_values(by="r2_score", ascending=False)
grid_search.to_pickle("data/pkl/resultados_gridsearch.pkl")


In [ ]:
grid_search

,best_params,mae,mse,rmse,r2_score
boosting_type,gbdt,0.016778,0.001221,0.034942,0.868567
learning_rate,0.1,0.016778,0.001221,0.034942,0.868567
max_depth,10,0.016778,0.001221,0.034942,0.868567
min_child_samples,20,0.016778,0.001221,0.034942,0.868567
n_estimators,1000,0.016778,0.001221,0.034942,0.868567
num_leaves,31,0.016778,0.001221,0.034942,0.868567
objective,regression,0.016778,0.001221,0.034942,0.868567
reg_alpha,0,0.016778,0.001221,0.034942,0.868567
reg_lambda,0,0.016778,0.001221,0.034942,0.868567
subsample,0.7,0.016778,0.001221,0.034942,0.868567


In [8]:
grid_search_results = pd.read_pickle("data/pkl/resultados_gridsearch.pkl")

best_params = grid_search_results['best_params'].to_dict() # Transformar en dict para usar en el modelo
best_params

# Entrenar el modelo con los mejores parámetros
best_model = LGBMRegressor(random_state=42, **best_params)
best_model.fit(X_train, y_train.ravel())

y_pred = best_model.predict(X_test)

# Calculamos y mostramos las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE", rmse)
print("R²:", r2)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 582
[LightGBM] [Info] Number of data points in the train set: 1997, number of used features: 24
[LightGBM] [Info] Start training from score 0.143845
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [9]:
# Guardar el modelo para usar en streamlit
joblib.dump(best_model, 'data/pkl/best_lightgbm_model.pkl')



['data/pkl/best_lightgbm_model.pkl']

Neural Networks

In [3]:
df_encoded = pd.read_csv("data/df_processed_to_ML.csv")
X = df_encoded.drop("prices_per_night", axis=1)
y = df_encoded["prices_per_night"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reutilizar los mismos escaladores
x_scaler = joblib.load("data/pkl/x_scaler.pkl")
y_scaler = joblib.load("data/pkl/y_scaler.pkl")

X_train_scaled = x_scaler.transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_train_scaled = y_scaler.transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

In [4]:
import plotly.express as px
import plotly.graph_objects as go

model = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),  # Camada de entrada
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(units=8, activation='relu'),
    Dense(units=1)  # Camada de salida
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_test_scaled, y_test_scaled),
    epochs=100,
    batch_size=32,
    verbose=1
)

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss', line=dict(color='orange')))
fig.update_layout(title="Pérdida durante el entrenamiento y validación (Modelo 1)",
                  xaxis_title="Epochs", yaxis_title="Loss", template="plotly_white",
                  legend=dict(x=0.5, y=1, xanchor='center', orientation='h'))
fig.show()

test_loss = model.evaluate(X_test_scaled, y_test_scaled)

predictions = model.predict(X_test_scaled)

# Desescalar las predicciones
predictions_original = y_scaler.inverse_transform(predictions)


# Graficar el resultado del entrenamiento del modelo usando plotly
resultados_df = pd.DataFrame({
    "Valores Reales": y_test,
    "Predicciones": predictions_original.ravel() # predictions tiene 2D, ravel para transformar en 1D para la creación del df
})

fig = px.scatter(
    resultados_df,
    x="Valores Reales",
    y="Predicciones",
    title="Valores Reales vs Predicciones",
    template="plotly_white"
)

# Añadir una línea de referencia donde las predicciones coinciden perfectamente con los valores reales
fig.add_shape(
    type="line",
    x0=resultados_df["Valores Reales"].min(),
    y0=resultados_df["Valores Reales"].min(),
    x1=resultados_df["Valores Reales"].max(),
    y1=resultados_df["Valores Reales"].max(),
    line=dict(color="red", dash="dot")
)

fig.show()


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0083 - val_loss: 0.0038
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0041 - val_loss: 0.0035
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0033 - val_loss: 0.0035
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0037 - val_loss: 0.0031
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0029 - val_loss: 0.0029
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0027 - val_loss: 0.0032
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0028 - val_loss: 0.0030
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0024 - val_loss: 0.0032
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - loss: 0.0017   
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [5]:
# Metricas
def calculate_metrics(y_real, y_pred):
    mae = mean_absolute_error(y_real, y_pred)
    rmse = np.sqrt(mean_squared_error(y_real, y_pred))
    r2 = r2_score(y_real, y_pred)
    return mae, rmse, r2

mae, rmse, r2 = calculate_metrics(y_test, predictions_original.ravel())

print("Métricas de evaluación del primer modelo:")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")


Métricas de evaluación del primer modelo:
MAE: 11.34398599243164
RMSE: 18.962940358884545
R2: 0.7944872101940863


In [6]:
# # Guardar el modelo para usar en streamlit
model.save('data/pkl/simple_nn_model_V2.keras')

Neural Networks - Hiperparametrización

In [7]:
# Definir learning rates para probar
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]

early_stopping = EarlyStopping(
    monitor="val_loss",  # Monitorar la pierda en la validación
    patience=3,          # Número de epochs sin mejora antes de parar
    restore_best_weights=True  # Restaurar los pesos del mejor epoch
)

# Lista para almacenar los resultados
results_lr = []

# Loop para entrenar el modelo con diferentes learning rates
for lr in learning_rates:
    model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=8, activation='relu')
])
    
    # Compilar el modelo con el learning rate actual
    model.compile(optimizer=Adam(learning_rate=lr), loss='mean_absolute_error')
    
    # Entrenar el modelo (con pocas epochs para pruebas rápidas)
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=10,
        batch_size=32,
        callbacks=[early_stopping],
        verbose=0  # Silenciar la salida para facilitar la lectura
    )

    results_lr.append({
        "learning_rate": lr,
        "validation_loss": history.history["val_loss"][-1]
    })

# Crear un DataFrame con los resultados
results_lr = pd.DataFrame(results_lr)
results_lr


,learning_rate,validation_loss
0,0.0001,43.078331
1,0.0010,37.752090
2,0.0100,28.423079
3,0.1000,69.400269
4,0.5000,82.495003


In [8]:
fig = px.line(
    results_lr,
    x="learning_rate",
    y="validation_loss",
    title="Learning Rate vs Validation Loss",
    labels={"learning_rate": "Learning Rate", "validation_loss": "Validation Loss"},
    log_x=True,  # Escala logarítmica para mejor visualización
    template="plotly_white"
)
fig.show()

In [9]:
# Encontrar el mejor learning rate basado en validation_loss
best_lr = results_lr.loc[results_lr["validation_loss"].idxmin(), "learning_rate"]

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,# Incrementar para prevenir paradas prematuras
    restore_best_weights=True # Restaurar mejores pesos
)

# Entrenar el mejor modelo con el mejor learning rate
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(units=128, activation='relu'),
    Dropout(0.2),  # Añadir Dropout para evitar overfitting
    Dense(units=64, activation='relu'),
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    Dense(units=8, activation='relu'),
    Dense(units=1)
])    

model.compile(optimizer=Adam(learning_rate=best_lr), loss='mean_absolute_error')


# Entrenar nuevamente el mejor modelo
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,  # Más epochs para ajustar bien
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss', line=dict(color='orange')))
fig.update_layout(title="Pérdida durante el entrenamiento y validación (Mejor LR)",
                  xaxis_title="Epochs", yaxis_title="Pérdida (Loss)", template="plotly_white",
                  legend=dict(x=0.5, y=1, xanchor='center', orientation='h'))
fig.show()

test_loss = model.evaluate(X_test, y_test)
print(f"Pierda en el conjunto de teste: {test_loss}")

predictions = model.predict(X_test)

resultados_df = pd.DataFrame({
    "Valores Reales": y_test,
    "Predicciones": predictions.ravel()  # predictions tiene 2D, ravel para transformar en 1D para la creación del df
})

fig = px.scatter(
    resultados_df,
    x="Valores Reales",
    y="Predicciones",
    title="Valores Reales vs Predicciones (Mejor LR)",
    template="plotly_white"
)

# Añadir una línea de referencia donde las predicciones coinciden perfectamente con los valores reales
fig.add_shape(
    type="line",
    x0=resultados_df["Valores Reales"].min(),
    y0=resultados_df["Valores Reales"].min(),
    x1=resultados_df["Valores Reales"].max(),
    y1=resultados_df["Valores Reales"].max(),
    line=dict(color="red", dash="dot")
)

fig.show()

Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 46.4492 - val_loss: 33.9559
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.2753 - val_loss: 35.1345
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.4841 - val_loss: 26.7674
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.3234 - val_loss: 25.7654
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.4727 - val_loss: 24.4341
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.5214 - val_loss: 35.0546
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.3277 - val_loss: 30.3749
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.2222 - val_loss: 23.0761
Epoch 9/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27.1505 - val_loss: 31.4395
Epoch 10/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.4835 - val_loss: 23.6004
Epoch 11/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.2769 - val_loss: 29.8228
Epoch 12/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - loss: 22.3462
Pierda en el conjunto de teste: 21.909589767456055
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [10]:
# Llamar la función definida en el primer modelo
mae, rmse, r2 = calculate_metrics(y_test, predictions.ravel())

print("Métricas de evaluación del segundo modelo:")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")


Métricas de evaluación del segundo modelo:
MAE: 21.90959116744995
RMSE: 30.56522238000942
R2: 0.46607209865002086


In [11]:
# Guardar el modelo para usar en streamlit
model.save('data/pkl/complex_nn_model_V2.keras')